In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out.basics import create_dir, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.in_out import PointCloudDataSet

from tf_lab.in_out.basics import read_saved_epochs          
from tf_lab.data_sets.shape_net import snc_category_to_synth_id, load_all_point_clouds_under_folder

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_points = 2048
do_training = True
first_time_running = True
load_pretrained_model = False
load_epoch = None
seed = 42

In [5]:
max_training_epochs = 1000
loss = 'chamfer'
bneck_size = 512

class_name = raw_input('Give me the class type: ').lower()
syn_id = snc_category_to_synth_id()[class_name]

experiment_id = raw_input('Give me the experiment ID:').lower()

experiment_name = '_'.join([class_name, experiment_id, str(n_pc_points), 
                           'pts_bneck', str(bneck_size), loss, 'all_data'
                           ])

top_data_dir = '/orions4-zfs/projects/optas/DATA/'
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/testing_ae_settings', experiment_name)
create_dir(train_dir)

Give me the class type: chair
Give me the experiment ID:no_bnorm_rep_old


'/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/testing_ae_settings/chair_no_bnorm_rep_old_2048_pts_bneck_512_chamfer_all_data'

In [6]:
pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_points))
pclouds_path = osp.join(pclouds_path, syn_id)
train_data = load_all_point_clouds_under_folder(pclouds_path, n_threads=40, verbose=True)

6778 pclouds were loaded. They belong in 1 shape-classes.


In [7]:
from tf_lab.point_clouds.ae_templates import conv_architecture_ala_nips_17

In [8]:
nips_args = conv_architecture_ala_nips_17(n_pc_points)
nips_args['encoder_args']['b_norm'] = False
nips_args['decoder_args']['b_norm'] = False

In [11]:
if load_pretrained_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    if conf.train_dir != train_dir:
        conf.train_dir = train_dir
    conf.save(osp.join(train_dir, 'configuration'))
else:
    conf = Conf(
                n_input = [n_pc_points, 3],
                loss = loss,
                training_epochs = 1000,
                batch_size = 50,
                denoising = False,
                learning_rate = 0.0005,
                train_dir = train_dir,              
                loss_display_step = 1,
                saver_step = 20,
                z_rotate = False,
                encoder = nips_args['encoder'],
                decoder = nips_args['decoder'],
                encoder_args = nips_args['encoder_args'],
                decoder_args = nips_args['decoder_args']
               )    
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))

In [12]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

In [13]:
if load_pretrained_model:
    if load_epoch is None: # load last saved.
        saved_epochs = read_saved_epochs(conf.train_dir)
        load_epoch = saved_epochs[-1]        
    ae.restore_model(conf.train_dir, load_epoch, verbose=True)

In [14]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)    
    train_stats = ae.train(train_data, conf, log_file=fout)
    fout.close()

('Epoch:', '0001', 'training time (minutes)=', '0.1698', 'loss=', '0.003545784')
INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/testing_ae_settings/chair_no_bnorm_rep_old_2048_pts_bneck_512_chamfer_all_data/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '0.1537', 'loss=', '0.002017159')
('Epoch:', '0003', 'training time (minutes)=', '0.1536', 'loss=', '0.001657753')
('Epoch:', '0004', 'training time (minutes)=', '0.1542', 'loss=', '0.001480896')
('Epoch:', '0005', 'training time (minutes)=', '0.1549', 'loss=', '0.001348797')
('Epoch:', '0006', 'training time (minutes)=', '0.1555', 'loss=', '0.001285595')
('Epoch:', '0007', 'training time (minutes)=', '0.1560', 'loss=', '0.001166831')
('Epoch:', '0008', 'training time (minutes)=', '0.1595', 'loss=', '0.001103347')
('Epoch:', '0009', 'training time (minutes)=', '0.1612', 'loss=', '0.001033924')
('Epoch:', '0010', 'training time (minutes)=', '0.16